# Keras starter kit [full training set, UNet]

In [1]:
# import sys

# sys.path.append('/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4')
# sys.path.append('/kaggle/input/segmentation-models-pytorch/segmentation_models.pytorch-master')
# sys.path.append('/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch-master')
# sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
# sys.path.append("/kaggle/input/einops/einops-master")

## Setup

In [2]:

import numpy as np
import torch
import torch.nn as nn
import wandb
import torchvision
import datetime
import imageio
# import cupy
from sklearn.model_selection import KFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning
import segmentation_models_pytorch as smp
import pytorch_lightning as pl
import pytorch_lightning.callbacks.model_checkpoint
import pytorch_lightning.plugins
from skimage.transform import resize as resize_ski
from pytorch_lightning.strategies.ddp import DDPStrategy
from pytorch_lightning.loggers import WandbLogger
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder, DecoderBlock
from timm.models.resnet import resnet10t, resnet34d, resnet50d, resnet14t, seresnext26d_32x4d, seresnext50_32x4d
from timm.models.mvitv2 import mvitv2_base
import os
import torch.utils.data
from dataclasses import dataclass

from scipy.ndimage import distance_transform_edt

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import os,cv2
import gc
import sys
import matplotlib.patches as patches
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import pytorch_lightning as pl

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
import albumentations as A
from IPython.display import Video

# sys.path.append("/home/fummicc1/codes/competitions/kaggle-ink-detection")
# sys.path.append("/kaggle/input/resnet3d")
from resnet import generate_model
# from resnext import generate_model

pytorch_lightning.seed_everything(seed=42)
torch.set_float32_matmul_precision('high')


@dataclass
class CFG():
    # Data config
    # DATA_DIR = '/kaggle/input/vesuvius-challenge-ink-detection/'
    DATA_DIR = '/home/fummicc1/codes/competitions/kaggle-ink-detection'
    # DATA_DIR = '/home/fummicc1/codes/Kaggle/kaggle-ink-detection'
    BUFFER = 192 # Half-size of papyrus patches we'll use as model inputs
    CROP_SIZE = BUFFER * 2
    STRIDE = 192
    # Z_LIST = list(range(0, 20, 5)) + list(range(22, 34))  # Offset of slices in the z direction
    # Z_LIST = [20, 22, 24, 26] + list(range(28, 36)) + [36, 38]
    # Z_LIST = list(range(16, 48, 2))
    Z_LIST = list(range(16, 48, 2))
    # Z_LIST = list(range(0, 24, 8)) + list(range(24, 36, 2)) + list(range(36, 64, 10))
    Z_DIM = len(Z_LIST)  # Number of slices in the z direction. Max value is 64 - Z_START
    BATCH_Z_DIFF = None
    SHARED_HEIGHT = 4032  # Max height to resize all papyrii

    # Model config
    BATCH_SIZE = 32
    
    GPU_COUNT = 4

    device = torch.device("cuda")
    threshold = 0.55
    num_workers = 8
    exp = 1e-7
    mask_padding = SHARED_HEIGHT // 40

    num_epochs = 30
    lr = 1e-3
    lr_scheduler_name = "CosineAnnealingLR"
    eta_min_lr = 1e-5
    WANDB_NOTE = "新ベースライン"
    augmentation_names = [
        "Horizontal",
        "Vertical",
        "RandomScale",
        "Transpose",
        "RandomRotate",
        "ShiftScaleRotate",
        # "Blur",
        "GridDistortion"
        "MultiChannelNoise",
        "PadIfNeeded",
        "Resize",   
    ]
    loss1_alpha = 1.75
    loss1_beta = None
    
    loss2a_alpha = 1.5
    loss2a_beta = None
    
    loss2b_alpha = 1.5
    
    loss1_weight = 0.5
    loss2_weight = 0.5
    
    loss2a_weight = 0.5
    loss2b_weight = 0.5
    
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR
    optimizer = "AdamW"
    # loss1 = smp.losses.TverskyLoss(
    #     smp.losses.BINARY_MODE,
    #     log_loss=False,
    #     from_logits=True, 
    #     smooth=1e-7,
    #     alpha=loss1_alpha,
    #     beta=loss1_beta,
    # )
    loss1 = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([loss1_alpha]),
    )
    # loss2a = smp.losses.TverskyLoss(
    #     smp.losses.BINARY_MODE,
    #     log_loss=False,
    #     from_logits=True, 
    #     smooth=1e-7,
    #     alpha=loss2a_alpha,
    #     beta=loss2a_beta,
    # )
    loss2a = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([loss2a_alpha]),
    )
    loss2b = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([loss2b_alpha]),
    )
    noise_intensity = (0.000025, 0.00005)
    use_new_label_mask = True
    pretrained = False
    # MODEL_DEPTH = 50
    # MODEL_KIND = "KMS"
    MODEL_DEPTH = 34
    MODEL_KIND = "KM"
    
    exp_name = "070-gpu17"
    prev_exp_name = ""
    
def class2dict(c):
    return {attr: getattr(c, attr) for attr in dir(c) if not callable(getattr(c, attr)) and not attr.startswith("__")}

Global seed set to 42


In [3]:
import cupy as cp
xp = cp

delta_lookup = {
    "xx": xp.array([[1, -2, 1]], dtype=float),
    "yy": xp.array([[1], [-2], [1]], dtype=float),
    "xy": xp.array([[1, -1], [-1, 1]], dtype=float),
}

def operate_derivative(img_shape, pair):
    assert len(img_shape) == 2
    delta = delta_lookup[pair]
    fft = xp.fft.fftn(delta, img_shape)
    return fft * xp.conj(fft)

def soft_threshold(vector, threshold):
    return xp.sign(vector) * xp.maximum(xp.abs(vector) - threshold, 0)

def back_diff(input_image, dim):
    assert dim in (0, 1)
    r, n = xp.shape(input_image)
    size = xp.array((r, n))
    position = xp.zeros(2, dtype=int)
    temp1 = xp.zeros((r+1, n+1), dtype=float)
    temp2 = xp.zeros((r+1, n+1), dtype=float)
    
    temp1[position[0]:size[0], position[1]:size[1]] = input_image
    temp2[position[0]:size[0], position[1]:size[1]] = input_image
    
    size[dim] += 1
    position[dim] += 1
    temp2[position[0]:size[0], position[1]:size[1]] = input_image
    temp1 -= temp2
    size[dim] -= 1
    return temp1[0:size[0], 0:size[1]]


def forward_diff(input_image, dim):
    assert dim in (0, 1)
    r, n = xp.shape(input_image)
    size = xp.array((r, n))
    position = xp.zeros(2, dtype=int)
    temp1 = xp.zeros((r+1, n+1), dtype=float)
    temp2 = xp.zeros((r+1, n+1), dtype=float)
        
    size[dim] += 1
    position[dim] += 1

    temp1[position[0]:size[0], position[1]:size[1]] = input_image
    temp2[position[0]:size[0], position[1]:size[1]] = input_image
    
    size[dim] -= 1
    temp2[0:size[0], 0:size[1]] = input_image
    temp1 -= temp2
    size[dim] += 1
    return -temp1[position[0]:size[0], position[1]:size[1]]

def iter_deriv(input_image, b, scale, mu, dim1, dim2):
    g = back_diff(forward_diff(input_image, dim1), dim2)
    d = soft_threshold(g + b, 1 / mu)
    b = b + (g - d)
    L = scale * back_diff(forward_diff(d - b, dim2), dim1)
    return L, b

def iter_xx(*args):
    return iter_deriv(*args, dim1=1, dim2=1)

def iter_yy(*args):
    return iter_deriv(*args, dim1=0, dim2=0)

def iter_xy(*args):
    return iter_deriv(*args, dim1=0, dim2=1)

def iter_sparse(input_image, bsparse, scale, mu):
    d = soft_threshold(input_image + bsparse, 1 / mu)
    bsparse = bsparse + (input_image - d)
    Lsparse = scale * (d - bsparse)
    return Lsparse, bsparse

def denoise_image(input_image, iter_num=100, fidelity=150, sparsity_scale=10, continuity_scale=0.5, mu=1):
    image_size = xp.shape(input_image)
    #print("Initialize denoising")
    norm_array = (
        operate_derivative(image_size, "xx") + 
        operate_derivative(image_size, "yy") + 
        2 * operate_derivative(image_size, "xy")
    )
    norm_array += (fidelity / mu) + sparsity_scale ** 2
    b_arrays = {
        "xx": xp.zeros(image_size, dtype=float),
        "yy": xp.zeros(image_size, dtype=float),
        "xy": xp.zeros(image_size, dtype=float),
        "L1": xp.zeros(image_size, dtype=float),
    }
    g_update = xp.multiply(fidelity / mu, input_image)
    for i in tqdm(range(iter_num), total=iter_num):
        #print(f"Starting iteration {i+1}")
        g_update = xp.fft.fftn(g_update)
        if i == 0:
            g = xp.fft.ifftn(g_update / (fidelity / mu)).real
        else:
            g = xp.fft.ifftn(xp.divide(g_update, norm_array)).real
        g_update = xp.multiply((fidelity / mu), input_image)
        
        #print("XX update")
        L, b_arrays["xx"] = iter_xx(g, b_arrays["xx"], continuity_scale, mu)
        g_update += L
        
        #print("YY update")
        L, b_arrays["yy"] = iter_yy(g, b_arrays["yy"], continuity_scale, mu)
        g_update += L
        
        #print("XY update")
        L, b_arrays["xy"] = iter_xy(g, b_arrays["xy"], 2 * continuity_scale, mu)
        g_update += L
        
        #print("L1 update")
        L, b_arrays["L1"] = iter_sparse(g, b_arrays["L1"], sparsity_scale, mu)
        g_update += L
        
    g_update = xp.fft.fftn(g_update)
    g = xp.fft.ifftn(xp.divide(g_update, norm_array)).real
    
    g[g < 0] = 0
    g -= g.min()
    g /= g.max()
    return g

In [4]:
def is_in_masked_zone(location, mask):
    return mask[location[0], location[1]] > 0


In [5]:
def resize(img):
    current_height, current_width = img.shape    
    aspect_ratio = current_width / current_height
    if CFG.SHARED_HEIGHT is None:
        return img
    # new_height = CFG.SHARED_HEIGHT
    # pad_y = new_height - current_height
    # if pad_y > 0:
    #     # 元画像が小さい場合は解像度を大きくしないでpaddingをつける
    #     img = np.pad(img, [(0, pad_y), (0, 0)], constant_values=0)
    # else:
    # 既に十分でかい場合はリサイズする
    # 本当はpaddingしたいけど、メモリサイズが大きくなる
    if CFG.SHARED_HEIGHT > current_height:
        # 既に小さい場合はリサイズしない
        return img    
    new_height = CFG.SHARED_HEIGHT
    new_width = int(CFG.SHARED_HEIGHT * aspect_ratio)
    new_size = (new_width, new_height)
    # (W, H)の順で渡すが結果は(H, W)になっている
    img = cv2.resize(img, new_size)
    return img

def load_mask(split, index):     
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/mask.png", 0) // 255
    img = resize(img)
    return img


def load_labels(split, index):    
    suffix = "_new" if CFG.use_new_label_mask else ""
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/inklabels{suffix}.png", 0) // 255    
    img = resize(img)
    return img

In [6]:
def extract_subvolume(location, volume):
    global printed
    y = location[0]
    x = location[1]
    subvolume = volume[y-CFG.BUFFER:y+CFG.BUFFER, x-CFG.BUFFER:x+CFG.BUFFER, :].astype(np.float32)
    
    return subvolume

In [7]:
def load_volume(split, index):    
    # Load the 3d x-ray scan, one slice at a time
    all = sorted(glob.glob(f"{CFG.DATA_DIR}/{split}/{index}/surface_volume/*.tif"))
    z_slices_fnames = [all[i] for i in range(len(all)) if i in CFG.Z_LIST]
    assert len(z_slices_fnames) == CFG.Z_DIM
    z_slices = []
    for z, filename in  tqdm(enumerate(z_slices_fnames)):
        img = cv2.imread(filename, -1)        
        img = resize(img)
        # img = (img / (2 ** 8)).astype(np.uint8)
        img = img.astype(np.float32) / 255
        z_slices.append(img)
    return np.stack(z_slices, axis=-1)

In [8]:
def generate_locations_ds(volume, mask, label=None, skip_zero=False):
    is_in_mask_train = lambda x: is_in_masked_zone(x, mask)

    # Create a list to store train locations
    locations = []

    # Generate train locations
    volume_height, volume_width = volume.shape[:-1]

    for y in range(CFG.BUFFER, volume_height - CFG.BUFFER, CFG.STRIDE):
        for x in range(CFG.BUFFER, volume_width - CFG.BUFFER, CFG.STRIDE):
            if skip_zero and label is not None and np.all(label[y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER] == 0):
                # print(f"skip location at (y: {y}, x: {x})")
                continue
            if is_in_mask_train((y, x)):
                locations.append((y, x))

    # Convert the list of train locations to a PyTorch tensor
    locations_ds = np.stack(locations, axis=0)
    return locations_ds

In [9]:
import numpy as np
from albumentations.core.transforms_interface import ImageOnlyTransform

class MultichannelNoise(ImageOnlyTransform):

    def __init__(self, intensity=CFG.noise_intensity, always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.intensity = intensity

    def apply(self, img, **params):
        intensity = np.random.uniform(*self.intensity)
        noise = np.random.normal(loc=0, scale=intensity, size=img.shape)
        # print("img", img)
        img = img + noise        
        return np.clip(img, 0, 1).astype(np.float32) # クリップして0から255の範囲に保つ


In [10]:

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder

from albumentations.core.transforms_interface import ImageOnlyTransform

class SubvolumeDataset(Dataset):
    def __init__(self, locations, volume, labels, buffer, is_train: bool, return_location: bool = False):
        self.locations = locations
        self.volume = volume
        self.labels = labels        
        self.buffer = buffer
        self.is_train = is_train
        self.return_location = return_location

    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        label = None
        location = np.array(self.locations[idx])
        y, x = location[0], location[1]

        subvolume = extract_subvolume(location, self.volume)
        
        if self.labels is not None:
            label = self.labels[y - self.buffer:y + self.buffer, x - self.buffer:x + self.buffer]            
            label = np.stack([label], axis=-1)            
            
        # 段々meanは小さくなる
        mean = np.array([0.42 for i in range(0, CFG.Z_DIM)]).reshape(-1, 1, 1)
        # 段々stdは小さくなる
        std = np.array([0.28 for i in range(0, CFG.Z_DIM)]).reshape(-1, 1, 1)
        
        if self.is_train and label is not None:    
            # label = smooth_labels(label, alpha=0.1)                  
            transformed = A.Compose([
                A.ToFloat(max_value=255),
                A.HorizontalFlip(p=0.5) if "Horizontal" in CFG.augmentation_names else A.Compose([]),
                A.VerticalFlip(p=0.5) if "Vertical" in CFG.augmentation_names else A.Compose([]),  
                A.RandomScale(p=0.4, scale_limit=0.4) if "RandomScale" in CFG.augmentation_names else A.Compose([]),
                A.Transpose(p=0.5) if "Transpose" in CFG.augmentation_names else A.Compose([]), 
                A.RandomRotate90(p=0.5,) if "RandomRotate" in CFG.augmentation_names else A.Compose([]), 
                A.ShiftScaleRotate(p=0.7, scale_limit=0.4, rotate_limit=80) if "ShiftScaleRotate" in CFG.augmentation_names else A.Compose([]),
                A.MotionBlur(p=0.2) if "Blur" in CFG.augmentation_names else A.Compose([]),
                A.GridDistortion(num_steps=5, distort_limit=0.15, p=0.2) if "GridDistortion" in CFG.augmentation_names else A.Compose([]),                
                MultichannelNoise(
                    p=0.2,
                ) if "MultichannelNoise" in CFG.augmentation_names else A.Compose([]),
                A.PadIfNeeded(min_height=self.buffer * 2, min_width=self.buffer * 2) if "PadIfNeeded" in CFG.augmentation_names else A.Compose([]),
                A.Resize(height=self.buffer * 2, width=self.buffer * 2) if "Resize" in CFG.augmentation_names else A.Compose([]),
            ])(image=subvolume, mask=label)            
            subvolume = transformed["image"]
            label = transformed["mask"]
            subvolume = np.transpose(subvolume, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))
            subvolume = (subvolume - mean) / std       
        else:
            if label is None:
                subvolume = np.transpose(subvolume, (2, 0, 1))
                subvolume /= 255.
                subvolume = (subvolume - mean) / std
            else:
                # print("subvolume in val dataset (before aug)", subvolume, file=open("before-val-aug.log", "w")) 
                subvolume = np.transpose(subvolume, (2, 0, 1))
                label = np.transpose(label, (2, 0, 1))
                subvolume /= 255.
                subvolume = (subvolume - mean) / std
        # print("subvolume", subvolume)
        if self.return_location:
            return subvolume, location
        return subvolume, label        

In [11]:
class SmpUnetDecoder(nn.Module):
	def __init__(
		self,
		in_channel,
		skip_channel,
		out_channel,
	):
		super().__init__()
		self.center = nn.Identity()

		i_channel = [
			in_channel,
		] + out_channel[:-1]
		s_channel = skip_channel
		o_channel = out_channel
		block = [
			DecoderBlock(i, s, o, use_batchnorm=True, attention_type=None)
			for i, s, o in zip(i_channel, s_channel, o_channel)
		]
		self.block = nn.ModuleList(block)

	def forward(self, feature, skip):
		d = self.center(feature)
		decode = []
		for i, block in enumerate(self.block):
			s = skip[i]
			d = block(d, s)
			decode.append(d)

		last = d
		return last, decode

In [12]:
class PoolingAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv = nn.Conv3d(in_channels, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch, _, _, _, _ = x.size()
        attention_map = self.sigmoid(self.conv(x))
        return attention_map


In [13]:
class Decoder(nn.Module):
    def __init__(self, encoder_dims, upscale):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(encoder_dims[i]+encoder_dims[i-1], encoder_dims[i-1], 3, 1, 1, bias=False),
                nn.BatchNorm2d(encoder_dims[i-1]),
                nn.ReLU(inplace=True)
            ) for i in range(1, len(encoder_dims))])

        self.logit = nn.Conv2d(encoder_dims[0], 1, 1, 1, 0)
        self.up = nn.Upsample(scale_factor=upscale, mode="bilinear")

    def forward(self, feature_maps):
        for i in range(len(feature_maps)-1, 0, -1):
            # print("index:", i, feature_maps[i].shape)
            f_up = F.interpolate(feature_maps[i], scale_factor=2, mode="bilinear")
            f = torch.cat([feature_maps[i-1], f_up], dim=1)
            f_down = self.convs[i-1](f)
            feature_maps[i-1] = f_down

        x = self.logit(feature_maps[0])
        mask = self.up(x)
        return mask

In [14]:
class SegModel(nn.Module):
	def __init__(self,model_depth=CFG.MODEL_DEPTH):
		super().__init__()
		self.encoder = generate_model(model_depth=model_depth, n_input_channels=1)
		# self.decoder = Decoder(encoder_dims=[128, 256, 512, 1024], upscale=4)
		self.decoder = Decoder(encoder_dims=[64, 64, 128, 256, 512], upscale=2)
		
	def forward(self, x):
		if x.ndim==4:
			x=x[:,None]
			
		encoder = []
		# print("start encoder")
		e = self.encoder
		x = e.conv1(x)
		x = e.bn1(x)
		x = e.relu(x)
		encoder.append(x)
		# print("start encoder layet1")
		x = F.avg_pool3d(x, kernel_size=(1,2,2), stride=(1,2,2))
		x = e.layer1(x)
		encoder.append(x)
		# print("start encoder layet2")
		x = e.layer2(x)
		encoder.append(x)
		# print("start encoder layet3")
		x = e.layer3(x)
		encoder.append(x)
		# print("start encoder layet4")
		x = e.layer4(x)
		encoder.append(x)
		# print("end encoder")
		
		feat_maps_pooled = [torch.mean(f, dim=2) for f in encoder]
		# print(feat_maps_pooled[0].shape)
		pred_mask = self.decoder(feat_maps_pooled)
		# print("pred_mask", pred_mask.shape)
		return pred_mask
	
	def load_pretrained_weights(self, state_dict):
		# Convert 3 channel weights to single channel
		# ref - https://timm.fast.ai/models#Case-1:-When-the-number-of-input-channels-is-1
		conv1_weight = state_dict['conv1.weight']
		state_dict['conv1.weight'] = conv1_weight.sum(dim=1, keepdim=True)
		print(self.encoder.load_state_dict(state_dict, strict=False))

In [15]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder=SegModel()
        weight_path = f"r3d{CFG.MODEL_DEPTH}_{CFG.MODEL_KIND}_200ep.pt"
        if CFG.pretrained:
            self.encoder.load_pretrained_weights(torch.load(weight_path)["state_dict"])

    def forward(self, images:torch.Tensor):
        #image.shape=(b,C,H,W)
        if images.ndim==4:
            images=images[:,None]
        # images=normalization(images)
        output = self.encoder(images)
        return output

In [16]:
tc = torch
def TTA(x:tc.Tensor,model:nn.Module):
    #x.shape=(batch,c,h,w)
    shape=x.shape
    x=[x,*[tc.rot90(x,k=i,dims=(-2,-1)) for i in range(1,4)]]
    x=tc.cat(x,dim=0)
    _, x = model(x)
    x=x.reshape(4,shape[0], 1 ,*shape[2:])
    x=[tc.rot90(x[i],k=-i,dims=(-2,-1)) for i in range(4)]
    x=tc.stack(x,dim=0)
    return x.mean(0)

In [17]:
# ref - https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/discussion/397288
def fbeta_score(preds, targets, threshold, beta=0.5, smooth=1e-5):
    preds_t = torch.where(preds > threshold, 1.0, 0.0).float()
    y_true_count = targets.sum()
    
    ctp = preds_t[targets==1].sum()
    cfp = preds_t[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

In [18]:


class Model(pl.LightningModule):
    training_step_outputs = []
    validation_step_outputs = []
    test_step_outputs = [[], []]

    def __init__(self, **kwargs):
        super().__init__()

        self.model = Net()        

        self.loss1 = CFG.loss1
        self.loss2 = CFG.loss2a
        self.loss3 = CFG.loss2b

    def forward(self, image, stage):
        if stage != "train":
            # mask = TTA(image, self.model)
            mask = self.model(image)
        else:
            mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        subvolumes, labels = batch

        image, labels = subvolumes.float(), labels.float()        
        assert image.ndim == 4
        
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0
        
        # print("labels", labels.max(), labels.min())

        assert labels.max() <= 1.0 and labels.min() >= 0

        if stage == "train":
            logit2 = self.forward(image, stage)
            # loss = self.loss1(logit1, labels) * CFG.loss1_weight + CFG.loss2_weight * (self.loss2(logit2, labels) * CFG.loss2a_weight + self.loss3(logit2, labels) * CFG.loss2b_weight)
            loss = (self.loss2(logit2, labels) * CFG.loss2a_weight + self.loss3(logit2, labels) * CFG.loss2b_weight)
            logit = logit2
        else:
            logit = self.forward(image, stage)
            loss = self.loss2(logit, labels) * CFG.loss2a_weight + self.loss3(logit, labels) * CFG.loss2b_weight
        
        prob2 = torch.sigmoid(logit)

        pred_mask = (prob2 > CFG.threshold).float()
        
        # print("pred_mask", pred_mask)
        
        score = fbeta_score(pred_mask, labels, threshold=CFG.threshold)

        tp, fp, fn, tn = smp.metrics.get_stats(
            pred_mask.long(), labels.long(), mode="binary"
        )
        # cur_lr = self.lr_schedulers().get_last_lr()

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
            "score": score,
            # "lr": cur_lr,
        }

    def shared_epoch_end(self, outputs, stage):
        # aggregate step metics
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])
        # lr = outputs[0]["lr"]
        loss = torch.mean(torch.Tensor([x["loss"] for x in outputs]))
        fbeta_score = torch.mean(torch.Tensor([x["score"] for x in outputs]))

        per_image_iou = smp.metrics.iou_score(
            tp, fp, fn, tn, reduction="micro-imagewise"
        )

        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")

        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
            f"{stage}_loss": 10000 if loss.item() == 0 else loss.item(),
            f"{stage}_tp": tp.sum().int().item(),
            f"{stage}_fp": fp.sum().int().item(),
            f"{stage}_fn": fn.sum().int().item(),
            f"{stage}_tn": tn.sum().int().item(),
            f"{stage}_score": fbeta_score.item(),
            # "lr": lr,
        }

        self.log_dict(metrics, prog_bar=True, sync_dist=True)

    def training_step(self, batch, batch_idx):
        out = self.shared_step(batch, "train")
        self.training_step_outputs.append(out)
        return out

    def on_train_epoch_end(self):
        out = self.shared_epoch_end(self.training_step_outputs, "train")
        self.training_step_outputs.clear()
        return out

    def validation_step(self, batch, batch_idx):
        out = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(out)
        return out

    def on_validation_epoch_end(self):
        out = self.shared_epoch_end(self.validation_step_outputs, "valid")
        self.validation_step_outputs.clear()
        return out


    def test_step(self, batch, batch_idx):
        global predictions_map, predictions_map_counts

        patch_batch, loc_batch = batch

        loc_batch = loc_batch.long()
        patch_batch = patch_batch.float()
        predictions = self.forward(patch_batch, "test")
        predictions = predictions.sigmoid()
 
        predictions = torch.permute(predictions, (0, 2, 3, 1)).squeeze(dim=-1)
        predictions = (
            predictions.cpu().numpy()
        )
        loc_batch = loc_batch.cpu().numpy()
        
        self.test_step_outputs[0].extend(loc_batch)
        self.test_step_outputs[1].extend(predictions)        
        return loc_batch, predictions

    def on_test_epoch_end(self):
        global predictions_map, predictions_map_counts

        locs = np.array(self.test_step_outputs[0])
        preds = np.array(self.test_step_outputs[1])
        print("locs", locs.shape)
        print("preds", preds.shape)

        new_predictions_map = np.zeros_like(predictions_map[:, :, 0])
        new_predictions_map_counts = np.zeros_like(predictions_map_counts[:, :, 0])

        for (y, x), pred in zip(locs, preds):
            new_predictions_map[
                y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER
            ] += pred
            new_predictions_map_counts[
                y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER
            ] += 1
        
        new_predictions_map /= new_predictions_map_counts + CFG.exp
        new_predictions_map = new_predictions_map[:, :, np.newaxis]
        predictions_map = np.concatenate(
            [predictions_map, new_predictions_map], axis=-1
        )


    def configure_optimizers(self):
        # optimizer = optim.SGD(self.parameters(), lr=CFG.lr)
        optimizer = optim.AdamW(self.parameters(), lr=CFG.lr)
        
        if CFG.lr_scheduler_name == "ReduceLROnPlateau":
            scheduler = CFG.lr_scheduler(
                optimizer, mode="min", factor=0.75, patience=5,
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "monitor": "valid_loss"},
            }
        elif CFG.lr_scheduler_name == "CosineAnnealingLR":
            scheduler = CFG.lr_scheduler(
                optimizer, T_max=10, eta_min=CFG.eta_min_lr,
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": scheduler,
            }

In [19]:
class EnsembleModel:
    def __init__(self, test_loader, test_volume):
        super().__init__()
        self.test_loader = test_loader
        self.test_volume = test_volume
        self.list = []
        for fold in [1]:
            _model = Model.load_from_checkpoint(
                f"weights/weights_fold-{fold}.ckpt",               
            )
            trainer = pl.Trainer(
                accelerator="gpu",
                devices="1",
                enable_checkpointing=False,
            )

            self.list.append((_model, trainer))
    
    def forward(self):
        global predictions_map, predictions_map_counts
        predictions_map = np.empty_like(self.test_volume[:, :, 0])[:, :, np.newaxis].astype(np.float64)
        predictions_map_counts = np.empty_like(predictions_map).astype(np.uint8)
        for i, (model, trainer) in enumerate(self.list):
            model: Model = model
            model.test_step_outputs = [[], []]
            model.eval()
            trainer.test(
                model=model,
                dataloaders=self.test_loader,
                verbose=True,
            )
            self.list[i] = None
            gc.collect()
        predictions_map = predictions_map[:, :, 1:].mean(axis=-1, keepdims=True)

In [20]:


import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
from skimage.transform import resize as resize_ski
import pathlib
import gc

def compute_predictions_map(split, index):
    global predictions_map
    global predictions_map_counts
    
    print(f"Load data for {split}/{index}")

    test_volume = load_volume(split=split, index=index)
    test_mask = load_mask(split=split, index=index)    
    
    #get coord
    stride = CFG.BUFFER
    H,W,D  = test_volume.shape

    ##pad #assume H,W >size
    px, py = W % stride, H % stride
    if (px != 0) or (py != 0):
        px = stride - px
        py = stride - py
        test_volume = np.pad(test_volume, [(0, py), (0, px), (0, 0)], constant_values=0)
        test_mask = np.pad(test_mask, [(0, py), (0, px)], constant_values=0)  
        
    test_locations = generate_locations_ds(test_volume, test_mask)  
    
    print(f"{len(test_locations)} test locations (before filtering by mask)")

    # filter locations inside the mask
    test_locations = [loc for loc in test_locations if is_in_masked_zone(loc, test_mask)]
    
    print(f"{len(test_locations)} test locations (after filtering by mask)")

    test_ds = SubvolumeDataset(test_locations, test_volume, None, CFG.BUFFER, is_train=False, return_location=True)
    test_loader = DataLoader(test_ds, batch_size=CFG.BATCH_SIZE, num_workers=CFG.num_workers, pin_memory=True)        

    # # shape: (Y, X, C)
    predictions_map = np.zeros_like(test_volume[:, :, 0])[:, :, np.newaxis].astype(np.float64)
    predictions_map_counts = np.zeros_like(predictions_map).astype(np.uint8)

    print("test_volume.shape", test_volume.shape)
    print("predictions_map.shape", predictions_map.shape)

    print(f"Compute predictions")
    
    model = EnsembleModel(test_loader, test_volume)
    model.forward()
    del model
    del test_locations
    del test_loader
    del test_ds
    del test_volume
    del test_mask
    gc.collect()
    
    # print("predictions_map", predictions_map, file=open("predictions_map", "w"))
    return predictions_map[:H, :W, :]


In [21]:
# Fast run length encoding, from https://www.kaggle.com/code/hackerpoet/even-faster-run-length-encoder/script
# (H, W)の形式
def rle (img: np.ndarray, threshold: float):
    flat_img = img.flatten()
    flat_img = np.where(flat_img > threshold, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix

    return " ".join(map(str, sum(zip(starts_ix, lengths), ())))

In [22]:
def update_submission(predictions_map, index):
    rle_ = rle(predictions_map, threshold=CFG.threshold)
    print(f"{index}," + rle_, file=open('submission.csv', 'a'))
    # print(f"{index}," + rle_, file=open('/kaggle/working/submission.csv', 'a'))

In [23]:
predictions_map = None
predictions_map_counts = None
print("Id,Predicted", file=open('submission.csv', 'w'))
kind = "test"
folder = pathlib.Path(CFG.DATA_DIR) / kind
for p in list(folder.iterdir()):
    index = p.stem
    predictions_map = compute_predictions_map(split=kind, index=index)
    original_size = cv2.imread(CFG.DATA_DIR + f"/{kind}/{index}/mask.png", 0).shape[:2]
    resized_predictions_map = resize_ski(predictions_map, (original_size[0], original_size[1], 1), anti_aliasing=True).squeeze(axis=-1)
    print("original size", original_size)
    resized_predictions_map=xp.array(resized_predictions_map)
    resized_predictions_map=denoise_image(resized_predictions_map, iter_num=250)
    resized_predictions_map=resized_predictions_map.get()
    update_submission(resized_predictions_map, index)
    resized_predictions_map = np.where(resized_predictions_map >= CFG.threshold, 255, 0)
    plt.imsave(f"{index}_{str(CFG.threshold)}_{str(CFG.BUFFER)}.png", resized_predictions_map, cmap="gray")
    resized_predicitions_map = None
    gc.collect()

Load data for test/a


16it [00:00, 25.17it/s]


269 test locations (before filtering by mask)
269 test locations (after filtering by mask)
test_volume.shape (2880, 6336, 16)
predictions_map.shape (2880, 6336, 1)
Compute predictions


/home/fummicc1/anaconda3/envs/ink-detection-3_8/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/fummicc1/anaconda3/envs/ink-detection-3_8/lib/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fummicc1/anaconda3/envs/ink-detection-3_8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `

Testing: 0it [00:00, ?it/s]

...

261 test locations (before filtering by mask)
261 test locations (after filtering by mask)
test_volume.shape (4224, 4800, 16)
predictions_map.shape (4224, 4800, 1)
Compute predictions


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

locs (261, 2)
preds (261, 384, 384)
original size (5454, 6330)


100%|██████████| 250/250 [01:05<00:00,  3.83it/s]
